In [4]:
import math
import time
import numpy as np
from numpy import pi
from collections import namedtuple

import pybullet
import pybullet_data


In [5]:
# 创建GUI服务器
pybullet.connect(pybullet.GUI)
# 配置渲染选项
# 载入棋盘格子
# 添加额外的文件搜索路径
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
# 载入棋盘平面(相当于地面)
plane_id = pybullet.loadURDF("plane_transparent.urdf")
# 设置重力
pybullet.setGravity(0, 0, -9.8)
# 开启实时仿真
pybullet.setRealTimeSimulation(True)

# 机械臂在世界坐标系下的位置, 单位m, 格式[x, y, z]
arm_posi = [0, 0, 0]
# 机械臂姿态-四元数, 格式 [x, y, z, w]
arm_q =  [0, 0, 0, 1]
# 定义标志位
# 1. 开启机械臂连杆的自碰撞检测, 不检测与父连杆的碰撞
flags = pybullet.URDF_USE_SELF_COLLISION_EXCLUDE_PARENT 
# 2. 使用URDF模型里面的转动惯量
flags |= pybullet.URDF_USE_INERTIA_FROM_FILE 
# 载入机械臂URDF模型
arm_id = pybullet.loadURDF("TwoJointRobot_w_fixedJoints.urdf", arm_posi, arm_q, \
                           useFixedBase=True, flags=flags)


In [6]:
# 获取仿真环境下Body个数
print(pybullet.getNumBodies())


2


In [7]:
# 查看arm4dof的信息
print(pybullet.getBodyInfo(arm_id))


(b'base_link', b'TwoJointRobot')


In [8]:
def get_body_id_by_name(body_name):
    '''根据Body的名称查询ID号'''
    body_num = pybullet.getNumBodies()
    for body_idx in range(body_num):
        cur_body_name = pybullet.getBodyInfo(body_idx)[-1].decode("utf-8")
        if body_name == cur_body_name:
            return body_idx
    return None

# 机械臂的名称
arm_name  = "TwoJointRobot" 
# 根据机械臂名称查询在PyBullet里面的Body ID
arm_id = get_body_id_by_name(arm_name)
print(f"机械臂的ID：{arm_id}")


机械臂的ID：1


In [9]:
#查询机械臂基座的位姿
pybullet.getBasePositionAndOrientation(arm_id)


((0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0))

In [12]:
#重置机械臂基座的位姿
# 定义基坐标系新的位置
new_position = [0, 0.4, 0]
# 定义欧拉角
roll = 0
pitch = 0
yaw = 0
# 欧拉角转换为四元数
new_quaternion = pybullet.getQuaternionFromEuler([roll, pitch, yaw])
print(f"new_quaternion ={new_quaternion}")
# 重置基座位姿
pybullet.resetBasePositionAndOrientation(arm_id, \
                        new_position, new_quaternion)


new_quaternion =(0.0, 0.0, 0.0, 1.0)


In [13]:
# 获取基座的速度
pybullet.getBaseVelocity(arm_id)


((0.0, 0.0, 0.0), (0.0, 0.0, 0.0))

In [15]:
# 获取机械臂所有的关节数
joint_num = pybullet.getNumJoints(arm_id)
print(f"机械臂关节个数:{joint_num}")


机械臂关节个数:6
<built-in function getJointInfo>


In [17]:
#查看机器人信息
print("机器人的信息：")
for joint_index in range(joint_num):
    info_tuple = pybullet.getJointInfo(arm_id, joint_index)
    print(f"关节序号：{info_tuple[0]}\n\
            关节名称：{info_tuple[1]}\n\
            关节类型：{info_tuple[2]}\n\
            机器人第一个位置的变量索引：{info_tuple[3]}\n\
            机器人第一个速度的变量索引：{info_tuple[4]}\n\
            保留参数：{info_tuple[5]}\n\
            关节的阻尼大小：{info_tuple[6]}\n\
            关节的摩擦系数：{info_tuple[7]}\n\
            slider和revolute(hinge)类型的位移最小值：{info_tuple[8]}\n\
            slider和revolute(hinge)类型的位移最大值：{info_tuple[9]}\n\
            关节驱动的最大值：{info_tuple[10]}\n\
            关节的最大速度：{info_tuple[11]}\n\
            节点名称：{info_tuple[12]}\n\
            局部框架中的关节轴系：{info_tuple[13]}\n\
            父节点frame的关节位置：{info_tuple[14]}\n\
            父节点frame的关节方向：{info_tuple[15]}\n\
            父节点的索引，若是基座返回-1：{info_tuple[16]}\n\n")

机器人的信息：
关节序号：0
            关节名称：b'joint_1'
            关节类型：0
            机器人第一个位置的变量索引：7
            机器人第一个速度的变量索引：6
            保留参数：1
            关节的阻尼大小：0.0
            关节的摩擦系数：0.0
            slider和revolute(hinge)类型的位移最小值：-3.0
            slider和revolute(hinge)类型的位移最大值：3.0
            关节驱动的最大值：10000.0
            关节的最大速度：5.0
            节点名称：b'link_01_cyl'
            局部框架中的关节轴系：(0.0, 0.0, 1.0)
            父节点frame的关节位置：(0.0, 0.0, 0.075)
            父节点frame的关节方向：(0.0, 0.0, 0.0, 1.0)
            父节点的索引，若是基座返回-1：-1


关节序号：1
            关节名称：b'link_01'
            关节类型：4
            机器人第一个位置的变量索引：-1
            机器人第一个速度的变量索引：-1
            保留参数：0
            关节的阻尼大小：0.0
            关节的摩擦系数：0.0
            slider和revolute(hinge)类型的位移最小值：0.0
            slider和revolute(hinge)类型的位移最大值：-1.0
            关节驱动的最大值：0.0
            关节的最大速度：0.0
            节点名称：b'link_1'
            局部框架中的关节轴系：(0.0, 0.0, 0.0)
            父节点frame的关节位置：(0.0, 0.0, 0.0)
            父节点frame的关节方向：(0.0, 0.0, 0.0,

In [18]:
#寻找可以控制的关节
# 可以使用的关节
available_joints_indexes = [i for i in range(pybullet.getNumJoints(arm_id)) if pybullet.getJointInfo(arm_id, i)[2] != pybullet.JOINT_FIXED]

print([pybullet.getJointInfo(arm_id, i)[1] for i in available_joints_indexes])

[b'joint_1', b'joint_2']


In [20]:
# 创建一个nametuple类型
JointInfo = namedtuple("JointInfo", ["id", "name", "lower_limit",\
                        "upper_limit", "max_force", "max_velocity"])

joint_name_list=['joint_1', 'joint_2']
# 构造关节信息字典
joint_info_dict = {}
for joint_idx in range(joint_num):
    # 获取每个关节的信息
    joint_info = pybullet.getJointInfo(arm_id, joint_idx)
    joint_name = joint_info[1].decode("utf8")
    if joint_name in joint_name_list:
        print("关节名称: {}".format(joint_name))
        joint_id = joint_info[0]
        joint_lower_limit = joint_info[8]
        joint_upper_limit = joint_info[9]
        joint_max_force = joint_info[10]
        joint_max_velocity = joint_info[11]
        joint_info = JointInfo(joint_id, joint_name, joint_lower_limit,
                   joint_upper_limit, joint_max_force, joint_max_velocity)
        joint_info_dict[joint_name] = joint_info
        
print(joint_info_dict)





关节名称: joint_1
关节名称: joint_2
{'joint_1': JointInfo(id=0, name='joint_1', lower_limit=-3.0, upper_limit=3.0, max_force=10000.0, max_velocity=5.0), 'joint_2': JointInfo(id=3, name='joint_2', lower_limit=-3.0, upper_limit=3.0, max_force=10000.0, max_velocity=5.0)}


In [22]:
#获取可控关节的ID
for o in joint_name_list:
    # 受控关节ID
    joint_id = joint_info_dict[o].id
    # 获取关节的角度下限
    lowerb = joint_info_dict[o].lower_limit
    # 获取关节的角度上限
    upperb = joint_info_dict[o].upper_limit
    print(f"{o} : id={joint_id} " + \
      f"lowerb={lowerb:.4f} upperb={upperb:.4f}")
# 受控关节ID列表
control_joint_id_list = []
for joint_name in joint_name_list:
    joint_id = joint_info_dict[joint_name].id
    control_joint_id_list.append(joint_id)

print(control_joint_id_list)


joint_1 : id=0 lowerb=-3.0000 upperb=3.0000
joint_2 : id=3 lowerb=-3.0000 upperb=3.0000
[0, 3]


In [23]:
# 获取单个关节的状态
pybullet.getJointState(arm_id, joint_idx)


(0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)

In [26]:
# 重置关节状态
joint_idx = 3 # 关节ID
target_position = 0 # 关节目标弧度
target_velocity = 1 # 关节目标速度
# 重置关节状态
pybullet.resetJointState(arm_id, joint_idx, \
        target_position, target_velocity)


In [31]:
# 重置下关节角度
# T0时刻的关节位置跟速度都设置为0
pybullet.resetJointState(arm_id, joint_id, 0, 0)
# 让关节1按照特定速度旋转
target_velocity = 1.0 # 关节目标转速
velocity_gain = 1.0 # 速度的增益, Kd
joint_info = joint_info_dict["joint_1"] # 获取关节信息
pybullet.setJointMotorControl2(arm_id, joint_info.id,\
            pybullet.VELOCITY_CONTROL, targetVelocity=target_velocity, \
            velocityGain=velocity_gain, force=joint_info.max_force)


In [34]:
# 关节ID
joint_id = joint_info_dict["joint_1"].id 
target_position = math.pi/4 # 目标位置
target_velocity = 0.0 # 目标转速
position_gain = 0.01 # 位置增益(Kp)
velocity_gain = 0.3 # 速度增益(Kd)
# 最大输出力矩
max_force = joint_info_dict["joint_1"].max_force

# 控制多个旋转关节的角度
# 位置控制 制定速度为0，位置为关节目标角度
# 位置增益: Kp
pybullet.setJointMotorControl2(
    arm_id, joint_id, pybullet.POSITION_CONTROL,
    targetPosition=target_position, targetVelocity=0.0,
    positionGain=position_gain, velocityGain=velocity_gain, \
    force=max_force)


In [37]:
# 关节个数
ctl_joint_num = len(control_joint_id_list)
# 目标过节角度
target_position_list = [math.pi/2, -math.pi/2]
# 控制多个旋转关节的角度
# 注: 这里给所有的关节设置了同一组PD控制器的参数positionGains与velocityGains，
pybullet.setJointMotorControlArray(
    arm_id, control_joint_id_list,
    pybullet.POSITION_CONTROL,
    targetPositions=target_position_list,
    targetVelocities=[0]*ctl_joint_num,
    positionGains=[0.01]*ctl_joint_num,
    velocityGains=[0.3] *ctl_joint_num,
    forces=[20.0]*ctl_joint_num)


In [38]:
import numpy as np

class PVT_Spline():
    def __init__(self, name='None'):
        """
        :param: name
            the name of the PVT_spline object
        """
        super(self.__class__, self).__init__()
        self.name = name

    def PVT_splineTwoPoints(self, via_point_0, via_point_1):
        """
        :param: via_point_0: float
            the first via points
        :param: via_point_1: float
            the second via points
        """
        p0 = via_point_0[0]
        v0 = via_point_0[1]
        t0 = via_point_0[2]

        p1 = via_point_1[0]
        v1 = via_point_1[1]
        t1 = via_point_1[2]

        T = t1-t0
        d = p0
        c = v0
        b = (3*p1 - v1*T - 2*v0*T - 3*p0)/(T**2)
        a = (-2*p1 + v1*T + v0*T + 2*p0)/(T**3)
        PVT_param = [a, b, c, d]

        return PVT_param

    def PVT_spline(self, via_points):
        """
        :param: via_points: float, np.array
            a set of the np.array with size Nx3, where N is the length of via points
        """
        queue_len = via_points.shape[0]
        if queue_len < 2:
            assert('point num too small')

        PVT_param_queue = np.zeros((queue_len-1, 4))

        for i in range(0, queue_len-1):
            PVT_param_queue[i,:] = self.PVT_splineTwoPoints(via_points[i,:], via_points[i+1,:])
        
        return PVT_param_queue

    def PVT_getPVTValue(self, PVT_param_queue, via_points, t):
        """
        :param: PVT_param_queue: float, list
            a set of the list object
        :param: via_points: float, np.array
            a set of the n[].array with size Nx3, where N is the length of via points
        """
        t_start = via_points[0, 2]
        t_end = via_points[-1, 2]
        PVT_point = np.zeros((1, 3))

        if t <= t_start:
            PVT_point_tmp = [via_points[0,0], via_points[0, 1], t]
        elif t >= t_end:
            PVT_point_tmp = [via_points[-1,0], via_points[-1, 1], t]
        else:
            for i in range(1, via_points.shape[0]):
                if t < via_points[i, 2]:
                    a = PVT_param_queue[i-1, 0]
                    b = PVT_param_queue[i-1, 1]
                    c = PVT_param_queue[i-1, 2]
                    d = PVT_param_queue[i-1, 3]
                    t0 = via_points[i-1, 2]

                    p = a*(t-t0)**3 + b*(t-t0)**2 + c*(t-t0) + d
                    v = 3*a*(t-t0)**2 + 2*b*(t-t0) + c
                    PVT_point_tmp = [p, v, t]
                    break

        PVT_point[0,0] = PVT_point_tmp[0]
        PVT_point[0,1] = PVT_point_tmp[1]
        PVT_point[0,2] = PVT_point_tmp[2]
        return PVT_point


In [47]:
#在GUI界面显示str
Text_id=pybullet.addUserDebugText(
    text="base",
    textPosition=[0, 0, 0],
    textColorRGB=[1, 0, 0],
    textSize=2,
)

#在GUI界面绘制线条（可用于绘制坐标系）
froms = [[1, 1, 0], [-1, 1, 0], [-1, 1, 3], [1, 1, 3]]
tos = [[-1, 1, 0], [-1, 1, 3], [1, 1, 3], [1, 1, 0]]
for f, t in zip(froms, tos):
    Line_id=pybullet.addUserDebugLine(
        lineFromXYZ=f,
        lineToXYZ=t,
        lineColorRGB=[0, 1, 0],
        lineWidth=10
    )






In [49]:
#移除1个文字、线条
pybullet.removeUserDebugItem(Text_id)
#全部移除
pybullet.removeAllUserDebugItems()



In [52]:
pybullet.getCameraImage(480,320)

(480,
 320,
 array([[[162, 186, 224, 255],
         [162, 186, 224, 255],
         [162, 186, 224, 255],
         ...,
         [162, 186, 224, 255],
         [162, 186, 224, 255],
         [162, 186, 224, 255]],
 
        [[162, 186, 224, 255],
         [162, 186, 224, 255],
         [162, 186, 224, 255],
         ...,
         [162, 186, 224, 255],
         [162, 186, 224, 255],
         [162, 186, 224, 255]],
 
        [[162, 186, 224, 255],
         [162, 186, 224, 255],
         [162, 186, 224, 255],
         ...,
         [239, 239, 239, 255],
         [239, 239, 239, 255],
         [239, 239, 239, 255]],
 
        ...,
 
        [[239, 239, 239, 255],
         [239, 239, 239, 255],
         [239, 239, 239, 255],
         ...,
         [239, 239, 239, 255],
         [239, 239, 239, 255],
         [239, 239, 239, 255]],
 
        [[239, 239, 239, 255],
         [239, 239, 239, 255],
         [239, 239, 239, 255],
         ...,
         [239, 239, 239, 255],
         [239, 239, 239